In [1]:
import pandas as pd

In [15]:

# Sample data
data = {
    "Main ID": ["01", "01", "01", "01", "01"],
    "sub ID": ["A", "A", "A", "B", "B"],
    "subsub ID": [1, 2, 3, 1, 2],
    "rule 1": ["r1A", "r1A", "r1A", "r1B", "r1B"],
    "rule 2": ["r2A", "r2A", "r2A", "r2B", "r2B"],
    "rule 3": ["r3A", "r3A", "r3A", "r3B", "r3B"],
    "value 1": [1, 1, 3, 3, 3],
    "value 2": [10, 10, 30, 30, 30],
    "value 3": [100, 100, 300, 300, 300],
    "sumval 1": [1, 2, 3, 4, 5],
    "sumval 2": [3, 4, 5, 6, 7],
    "sumval 3": [2, 3, 4, 5, 6],
    "valid": [True, True, True, True, True]
}

# Create DataFrame
df = pd.DataFrame(data)

# Adding a new column 'weight' to the original dataframe
df['weight'] = df['sumval 2'] + df['sumval 3']

In [3]:

# Adjusting the aggregation function to include the new 'weight' column
def aggregate_dataframe_with_weight(df):
    # Group by 'Main ID' and 'rule 1', 'rule 2', 'rule 3'
    grouped = df.groupby(['Main ID', 'rule 1', 'rule 2', 'rule 3'])

    # Aggregate data
    aggregated_df = grouped.agg({
        'sub ID': lambda x: list(x),
        'subsub ID': lambda x: list(x),
        'value 1': 'first',
        'value 2': 'first',
        'value 3': 'first',
        'sumval 1': 'sum',
        'sumval 2': 'sum',
        'sumval 3': 'sum',
        'weight': 'sum'
    }).reset_index()

    # Combining sub ID and subsub ID into tuples
    aggregated_df['sub IDs'] = aggregated_df.apply(lambda row: list(zip(row['sub ID'], row['subsub ID'])), axis=1)

    # Dropping separate sub ID and subsub ID columns
    aggregated_df.drop(columns=['sub ID', 'subsub ID'], inplace=True)

    return aggregated_df

# Aggregate the dataframe with the new 'weight' column
aggregated_df_with_weight = aggregate_dataframe_with_weight(df)
aggregated_df_with_weight



,Main ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,weight,sub IDs
0,01,r1A,r2A,r3A,1,10,100,6,12,9,21,"[(A, 1), (A, 2), (A, 3)]"
1,01,r1B,r2B,r3B,3,30,300,9,13,11,24,"[(B, 1), (B, 2)]"


In [4]:
# Aggregate the dataframe with the corrected function
aggregated_df_corrected = aggregate_dataframe_with_weight(df)
aggregated_df_corrected

,Main ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,weight,sub IDs
0,01,r1A,r2A,r3A,1,10,100,6,12,9,21,"[(A, 1), (A, 2), (A, 3)]"
1,01,r1B,r2B,r3B,3,30,300,9,13,11,24,"[(B, 1), (B, 2)]"


In [8]:
# Adding 'result' column to the aggregated dataframe
aggregated_df_corrected['agg_result'] = [42, 24]

# Merging the original dataframe with the aggregated dataframe on 'Main ID' and 'rule 1'
merged_df = df.merge(aggregated_df_corrected[['Main ID', 'rule 1', 'weight', 'agg_result']], on=['Main ID', 'rule 1'], suffixes=('', '_agg'))

# Calculate the proportional 'result' for each row in the original dataframe
merged_df['result'] = merged_df.apply(lambda row: row['agg_result'] * row['weight'] / row['weight_agg'], axis=1)


,Main ID,sub ID,subsub ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,valid,weight,result
0,01,A,1,r1A,r2A,r3A,1,10,100,1,3,2,True,5,10.0
1,01,A,2,r1A,r2A,r3A,1,10,100,2,4,3,True,7,14.0
2,01,A,3,r1A,r2A,r3A,3,30,300,3,5,4,True,9,18.0
3,01,B,1,r1B,r2B,r3B,3,30,300,4,6,5,True,11,11.0
4,01,B,2,r1B,r2B,r3B,3,30,300,5,7,6,True,13,13.0


## add validity

In [14]:
# Adjusting the aggregation function to include only rows where valid == True
def aggregate_dataframe_with_validity(df):
    # Filter the dataframe for rows where 'valid' is True
    valid_df = df[df['valid'] == True]

    # Group by 'Main ID' and 'rule 1', 'rule 2', 'rule 3'
    grouped = valid_df.groupby(['Main ID', 'rule 1', 'rule 2', 'rule 3'])

    # Aggregate data
    aggregated_df = grouped.agg({
        'sub ID': lambda x: list(x),
        'subsub ID': lambda x: list(x),
        'value 1': 'first',
        'value 2': 'first',
        'value 3': 'first',
        'sumval 1': 'sum',
        'sumval 2': 'sum',
        'sumval 3': 'sum',
        'weight': 'sum'
    }).reset_index()

    # Combining sub ID and subsub ID into tuples
    aggregated_df['sub IDs'] = aggregated_df.apply(lambda row: list(zip(row['sub ID'], row['subsub ID'])), axis=1)

    # Dropping separate sub ID and subsub ID columns
    aggregated_df.drop(columns=['sub ID', 'subsub ID'], inplace=True)

    return aggregated_df

In [17]:
# Sample data with an additional 'valid' column
data_with_validity = {
    "Main ID": ["02", "02", "02"],
    "sub ID": ["A", "A", "A"],
    "subsub ID": [1, 2, 3],
    "rule 1": ["r1A", "r1A", "r1A"],
    "rule 2": ["r2A", "r2A", "r2A"],
    "rule 3": ["r3A", "r3A", "r3A"],
    "value 1": [1, 1, 1],
    "value 2": [10, 10, 10],
    "value 3": [100, 100, 100],
    "sumval 1": [1, 2, 2],
    "sumval 2": [3, 4, 4],
    "sumval 3": [2, 3, 3],
    "valid": [True, True, False],
    "weight": [5, 7, 7]
}

# Create DataFrame
df_with_validity = pd.DataFrame(data_with_validity)

In [20]:
# Aggregate the dataframe with the new validity logic
aggregated_df_with_validity = aggregate_dataframe_with_validity(df_with_validity)

# Adding 'result' column to the aggregated dataframe
aggregated_df_with_validity['agg_result'] = 36  # Total result to be distributed

aggregated_df_with_validity

,Main ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,weight,sub IDs,agg_result
0,02,r1A,r2A,r3A,1,10,100,3,7,5,12,"[(A, 1), (A, 2)]",36


In [26]:
# Merging the original dataframe with the aggregated dataframe on 'Main ID' and 'rule 1'
# Note: Only valid rows should be considered for redistribution
merged_df_valid_only = df_with_validity.merge(
    aggregated_df_with_validity[['Main ID', 'rule 1', 'weight', 'agg_result']], 
    on=['Main ID', 'rule 1'], 
    suffixes=('', '_agg')
)

# Calculate the proportional 'result' for each valid row in the original dataframe
merged_df_valid_only['result'] = merged_df_valid_only.apply(
    lambda row: row['agg_result'] * row['weight'] / row['weight_agg'] if row["valid"] else pd.NA, axis=1
)

merged_df_valid_only

,Main ID,sub ID,subsub ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,valid,weight,weight_agg,agg_result,result
0,02,A,1,r1A,r2A,r3A,1,10,100,1,3,2,True,5,12,36,15.0
1,02,A,2,r1A,r2A,r3A,1,10,100,2,4,3,True,7,12,36,21.0
2,02,A,3,r1A,r2A,r3A,1,10,100,2,4,3,False,7,12,36,<NA>


## add different granularity

In [29]:
# Sample data with new rules and values
data_with_rules = {
    "Main ID": ["03", "03", "03", "03", "03"],
    "sub ID": ["C", "C", "C", "D", "D"],
    "subsub ID": [1, 2, 3, 1, 2],
    "rule 1": ["r1C", "r1C", "r1C", "r1D", "r1D"],
    "rule 2": ["r2C", "r2D", "r2E", "r2F", "r2F"],
    "rule 3": ["r3C", "r3C", "r3C", "r3D", "r3D"],
    "value 1": [1, 1, 1, 3, 3],
    "value 2": [10, 20, 30, 40, 40],
    "value 3": [100, 100, 100, 300, 300],
    "sumval 1": [1, 2, 3, 4, 5],
    "sumval 2": [3, 4, 5, 6, 7],
    "sumval 3": [2, 3, 4, 5, 6],
    "valid": [True, True, True, True, True],
    "weight": [5, 7, 9, 11, 13]
}

# Create DataFrame
df_with_rules = pd.DataFrame(data_with_rules)
df_with_rules

,Main ID,sub ID,subsub ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,valid,weight
0,03,C,1,r1C,r2C,r3C,1,10,100,1,3,2,True,5
1,03,C,2,r1C,r2D,r3C,1,20,100,2,4,3,True,7
2,03,C,3,r1C,r2E,r3C,1,30,100,3,5,4,True,9
3,03,D,1,r1D,r2F,r3D,3,40,300,4,6,5,True,11
4,03,D,2,r1D,r2F,r3D,3,40,300,5,7,6,True,13


In [30]:
# Adjusted aggregation function with specific logic for summing up value 2 and value 3 based on unique rule 2 and rule 3 values

def aggregate_dataframe_with_specific_rules(df):
    # Group by 'Main ID' and 'rule 1'
    grouped = df.groupby(['Main ID', 'rule 1'])

    # Custom aggregation function
    def custom_agg(x):
        if x.nunique() == 1:
            return x.iloc[0]
        else:
            return x.sum()

    # Aggregate data
    aggregated_df = grouped.agg({
        'sub ID': lambda x: list(x),
        'subsub ID': lambda x: list(x),
        'rule 2': lambda x: list(set(x)),
        'rule 3': lambda x: list(set(x)),
        'value 1': 'first',
        'value 2': custom_agg,
        'value 3': custom_agg,
        'sumval 1': 'sum',
        'sumval 2': 'sum',
        'sumval 3': 'sum',
        'valid': 'first',  # Assuming we only need to check if at least one row is valid
        'weight': 'sum'
    }).reset_index()

    # Combining sub ID and subsub ID into tuples
    aggregated_df['sub IDs'] = aggregated_df.apply(lambda row: list(zip(row['sub ID'], row['subsub ID'])), axis=1)

    # Dropping separate sub ID and subsub ID columns
    aggregated_df.drop(columns=['sub ID', 'subsub ID'], inplace=True)

    return aggregated_df



In [31]:
# Aggregate the dataframe with the specific rules logic
aggregated_df_with_specific_rules = aggregate_dataframe_with_specific_rules(df_with_rules)
aggregated_df_with_specific_rules

,Main ID,rule 1,rule 2,rule 3,value 1,value 2,value 3,sumval 1,sumval 2,sumval 3,valid,weight,sub IDs
0,03,r1C,"[r2E, r2D, r2C]",[r3C],1,60,100,6,12,9,True,21,"[(C, 1), (C, 2), (C, 3)]"
1,03,r1D,[r2F],[r3D],3,40,300,9,13,11,True,24,"[(D, 1), (D, 2)]"


In [32]:
def check_granularity(df):
    # Grouping by Main ID and rule 1 to see if rule 2 or rule 3 has different granularity
    grouped = df.groupby(['Main ID', 'rule 1'])

    for (main_id, rule1), group in grouped:
        unique_rule2 = group['rule 2'].nunique()
        unique_rule3 = group['rule 3'].nunique()

        if unique_rule2 > 1 or unique_rule3 > 1:
            print(f"Different granularity found for Main ID: {main_id}, Rule 1: {rule1}")
            print(f"Rows where this holds:")
            print(group[['sub ID', 'subsub ID', 'rule 2', 'rule 3']], "\n")

# Testing the function with the provided dataframe
check_granularity(df_with_rules)



Different granularity found for Main ID: 03, Rule 1: r1C
Rows where this holds:
  sub ID  subsub ID rule 2 rule 3
0      C          1    r2C    r3C
1      C          2    r2D    r3C
2      C          3    r2E    r3C 

